In [5]:
import os
import geopandas as gpd
import rasterio
from rasterio import features
from scipy.ndimage import distance_transform_edt
from osgeo import gdal
import numpy as np
from pathlib import Path
import time

In [6]:
# --- KONFIGURATION & PFADE ---
try:
    SCRIPT_DIR = Path(__file__).parent
except NameError:
    SCRIPT_DIR = Path.cwd()

PROJECT_ROOT = SCRIPT_DIR.parent
DATA_DIR = PROJECT_ROOT / 'data'
OUTPUT_DIR = PROJECT_ROOT / 'output'

# Input Files
DEM_PATH = OUTPUT_DIR / "dem_10m_graubuenden.tif"
BOUNDARIES_PATH = DATA_DIR / "swissBOUNDARIES3D_1_5_LV95_LN02.gpkg"
TLM_PATH = DATA_DIR / "SWISSTLM3D_2025.gpkg"
AREAL_PATH = DATA_DIR / "arealstatistik_2056.gpkg"

# Output Files
SLOPE_PATH = OUTPUT_DIR / "slope_10m_graubuenden.tif"
FOREST_RASTER_PATH = OUTPUT_DIR / "forest_10m_graubuenden.tif"
ROADS_DIST_PATH = OUTPUT_DIR / "distance_roads_10m.tif"
SETTLEMENT_DIST_PATH = OUTPUT_DIR / "distance_settlements_10m.tif"
WATER_MASK_PATH = OUTPUT_DIR / "water_mask_10m.tif"
PREY_RASTER_PATH = OUTPUT_DIR / "prey_10m.tif"

In [7]:
# --- HILFSFUNKTIONEN ---

def safe_delete(path):
    """Versucht eine Datei sicher zu löschen, um PermissionErrors zu vermeiden."""
    if path.exists():
        try:
            os.remove(path)
        except PermissionError:
            print(f"⚠️ ACHTUNG: Konnte {path.name} nicht löschen. Datei ist gesperrt!")
            print("   -> Bitte schliesse QGIS oder andere Programme, die die Datei nutzen.")
            raise PermissionError(f"File locked: {path}")
        except Exception as e:
            print(f"Fehler beim Löschen von {path.name}: {e}")

def get_graubuenden_boundary():
    print("-> Lade Kantonsgrenze Graubünden...")
    try:
        gdf = gpd.read_file(BOUNDARIES_PATH, layer='TLM_KANTONSGEBIET')
        
        # Filter logic
        if 'NAME' in gdf.columns:
            gdf_gr = gdf[gdf['NAME'].isin(['Graubünden', 'Grigioni', 'Grischun'])]
        elif 'name' in gdf.columns:
            gdf_gr = gdf[gdf['name'].isin(['Graubünden', 'Grigioni', 'Grischun'])]
        elif 'kantonsnummer' in gdf.columns:
            gdf_gr = gdf[gdf['kantonsnummer'] == 18]
        else:
            print("Warnung: Konnte Graubünden nicht filtern. Nehme alles.")
            gdf_gr = gdf

        # Geometrie vereinigen (Fix für Deprecation Warning)
        try:
            return gdf_gr.geometry.union_all()
        except AttributeError:
            return gdf_gr.geometry.unary_union # Fallback für ältere Geopandas Versionen
            
    except Exception as e:
        print(f"Fehler beim Laden der Grenzen: {e}")
        return None

def simple_rasterize(gdf, reference_path, output_path):
    """Rasterisiert Features als 1 (Binär), alles andere 0."""
    safe_delete(output_path) # Platz machen
    
    with rasterio.open(reference_path) as src:
        meta = src.meta.copy()
        meta.update(dtype=rasterio.uint8, count=1, nodata=0)
        shape = src.shape
        transform = src.transform
        
    print(f"   ... Rasterisiere {len(gdf)} Objekte (Binär)...")
    if len(gdf) > 0:
        shapes = ((geom, 1) for geom in gdf.geometry)
        arr = features.rasterize(shapes=shapes, out_shape=shape, transform=transform, fill=0, default_value=1, dtype=rasterio.uint8)
    else:
        print("   ⚠️ Keine Features gefunden! Erstelle leeres Raster.")
        arr = np.zeros(shape, dtype=rasterio.uint8)
    
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(arr, 1)
    print(f"   ✅ Gespeichert: {output_path.name}")

def rasterize_and_distance(gdf, reference_path, output_path, burn_value=1):
    """Rasterisiert und berechnet Euklidische Distanz."""
    safe_delete(output_path)
    
    with rasterio.open(reference_path) as src:
        meta = src.meta.copy()
        shape = src.shape
        transform = src.transform
        cell_size = src.res[0]

    print(f"   ... Rasterisiere {len(gdf)} Objekte...")
    if len(gdf) > 0:
        shapes = ((geom, burn_value) for geom in gdf.geometry)
        binary = features.rasterize(shapes=shapes, out_shape=shape, transform=transform, default_value=burn_value, dtype=rasterio.uint8)
    else:
        binary = np.zeros(shape, dtype=rasterio.uint8)

    print("   ... Berechne Distanz...")
    mask = np.logical_not(binary) 
    dist_meters = distance_transform_edt(mask) * cell_size

    meta.update(dtype=rasterio.float32, count=1, nodata=-9999)
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(dist_meters.astype(rasterio.float32), 1)
    print(f"   ✅ Gespeichert: {output_path.name}")

In [8]:
# ==========================================
# HAUPTPROGRAMM
# ==========================================

gr_geometry = get_graubuenden_boundary()
if gr_geometry is None: exit()

# 1. SLOPE
print("\n=== 1. SLOPE ===")
if not SLOPE_PATH.exists():
    gdal.DEMProcessing(str(SLOPE_PATH), str(DEM_PATH), "slope", options=gdal.DEMProcessingOptions(computeEdges=True))
    print("✅ Slope berechnet.")
else: print("ℹ️ Slope existiert bereits.")

# 2. WALD
print("\n=== 2. WALD (Deckung) ===")
# Lade Forest Types
gdf_land = gpd.read_file(TLM_PATH, layer='tlm_bb_bodenbedeckung', mask=gr_geometry)
forest_types = ['Wald', 'Gebueschwald', 'Wald offen', 'Gehoelzflaeche']
gdf_forest = gdf_land[gdf_land['objektart'].isin(forest_types)]
simple_rasterize(gdf_forest, DEM_PATH, FOREST_RASTER_PATH)

# 3. WASSER (Hindernis)
print("\n=== 3. WASSER (Seen) ===")
# Gleicher Layer, andere Objektart
gdf_water = gdf_land[gdf_land['objektart'] == 'Stehende Gewaesser']
simple_rasterize(gdf_water, DEM_PATH, WATER_MASK_PATH)

# 4. STRASSEN (Störung)
print("\n=== 4. STRASSEN (Störung) ===")
gdf_roads = gpd.read_file(TLM_PATH, layer='tlm_strassen_strasse', mask=gr_geometry)
# Filter für grosse Strassen
heavy_types = ['Autobahn', 'Autostrasse', '10m Strasse', '8m Strasse', '6m Strasse', 'Hauptstrasse', 'Verbindungsstrasse', 'Einfahrt', 'Ausfahrt', 'Zufahrt']
# Filtere Tunnel raus
gdf_surf = gdf_roads[gdf_roads['objektart'].isin(heavy_types) & ~gdf_roads['kunstbaute'].isin(['Tunnel', 'Unterfuehrung'])]
if not gdf_surf.empty:
    rasterize_and_distance(gdf_surf, DEM_PATH, ROADS_DIST_PATH)

# 5. SIEDLUNGEN (Störung)
print("\n=== 5. SIEDLUNGEN (Störung) ===")
AS_COLUMN = 'AS18_72' # Nutze AS18 für Graubünden
gdf_areal = gpd.read_file(AREAL_PATH, layer='arealstatistik_all', mask=gr_geometry)

if AS_COLUMN in gdf_areal.columns:
    # Codes 1-13 sind Siedlung/Gebäude
    gdf_settle = gdf_areal[gdf_areal[AS_COLUMN].isin(list(range(1, 14)))]
    if not gdf_settle.empty:
        print("   ... Erstelle Buffer (150m) um Siedlungen...")
        gdf_settle_poly = gpd.GeoDataFrame(geometry=gdf_settle.buffer(150)) 
        rasterize_and_distance(gdf_settle_poly, DEM_PATH, SETTLEMENT_DIST_PATH)

# 6. BEUTE / PREY (Nahrung)
print("\n=== 6. PREY (Alpwirtschaft) ===")
if AS_COLUMN in gdf_areal.columns:
    # Codes 45-49: Alpwiesen & Weiden
    target_codes = [45, 46, 47, 48, 49]
    print(f"   Filtere nach Codes: {target_codes}")
    gdf_prey = gdf_areal[gdf_areal[AS_COLUMN].isin(target_codes)]
    print(f"   -> {len(gdf_prey)} Punkte gefunden.")
    if not gdf_prey.empty:
        # Kein Buffer nötig, wird im Analyse-Skript geglättet (Gaussian Blur)
        simple_rasterize(gdf_prey, DEM_PATH, PREY_RASTER_PATH)
    else:
        print("⚠️ Keine Prey-Daten gefunden.")

print("\n✅ PREPROCESSING COMPLETED.")

-> Lade Kantonsgrenze Graubünden...

=== 1. SLOPE ===
ℹ️ Slope existiert bereits.

=== 2. WALD (Deckung) ===
   ... Rasterisiere 58825 Objekte (Binär)...
   ✅ Gespeichert: forest_10m_graubuenden.tif

=== 3. WASSER (Seen) ===
   ... Rasterisiere 3593 Objekte (Binär)...
   ✅ Gespeichert: water_mask_10m.tif

=== 4. STRASSEN (Störung) ===
   ... Rasterisiere 5647 Objekte...
   ... Berechne Distanz...
   ✅ Gespeichert: distance_roads_10m.tif

=== 5. SIEDLUNGEN (Störung) ===
   ... Erstelle Buffer (150m) um Siedlungen...
   ... Rasterisiere 6641 Objekte...
   ... Berechne Distanz...
   ✅ Gespeichert: distance_settlements_10m.tif

=== 6. PREY (Alpwirtschaft) ===
   Filtere nach Codes: [45, 46, 47, 48, 49]
   -> 160112 Punkte gefunden.
   ... Rasterisiere 160112 Objekte (Binär)...
   ✅ Gespeichert: prey_10m.tif

✅ PREPROCESSING COMPLETED.
